In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.preprocessing import KernelCenterer
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import KernelPCA
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import MiniBatchSparsePCA
from sklearn.decomposition import SparsePCA
from sklearn.decomposition import SparseCoder
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import train_test_split
from sklearn.metrics import pairwise_distances_argmin
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.cluster import AffinityPropagation
from markdown import markdown
import numpy as np
import hvplot.pandas
import pandas as pd





### Analysis of Birds bones and living habits data set
## An attempt at species categorization using sklearn Kmeans Unsupervised learning.
https://www.kaggle.com/datasets/zhangjuefei/birds-bones-and-living-habits


https://scikit-learn.org/stable/modules/clustering.html#k-means


https://github.com/oohtmeel1/Unsup2 < Project Repo>



![Pidgeon Skeleton](https://s-media-cache-ak0.pinimg.com/564x/cb/cb/41/cbcb41cb2c1db6c32c1e869be2172c3b.jpg) Image courtesy of kaggle





Species identification is an important part of conservation efforts. This project attempts to use the Birds bones and living habits data set to label species based on skeletal features.The idea is that birds skeletal structure should be varied enough to allow for easy and simple identification. 
The data labels are:


<b>*SW* :Swimming \
*W*  :Wading \
*T*  :Terrestrial \
*R*  :Raptors \
*P*  :Scansorial_birds \
*SO* :Singing Birds </b>

These will be how the data is clustered. No two bird species are exactly alike, so given enough data points are provided this should work. 

Since the data set seemed very distinct (An emu is much larger than say a chicken) K means felt like a very good choice. The basic idea here is: K centroids will be chosen, the model will be fit and when the centroids stop moving is when we are done. Even though K means is supposed to be non-deterministic, more often than not a pidgeon and emu should not be clustered together. 


After the data was read in, the next step was to take a look. Right away it can be noted that the ID column should
be dropped, it serves no purpose. 


|    |   id |   huml |   humw |   ulnal |   ulnaw |   feml |   femw |   tibl |   tibw |   tarl |   tarw | type   |
|---:|-----:|-------:|-------:|--------:|--------:|-------:|-------:|-------:|-------:|-------:|-------:|:-------|
|  0 |    0 |  80.78 |   6.68 |   72.01 |    4.88 |  41.81 |   3.7  |   5.5  |   4.03 |  38.7  |   3.84 | SW     |
|  1 |    1 |  88.91 |   6.63 |   80.53 |    5.59 |  47.04 |   4.3  |  80.22 |   4.51 |  41.5  |   4.01 | SW     |
|  2 |    2 |  79.97 |   6.37 |   69.26 |    5.28 |  43.07 |   3.9  |  75.35 |   4.04 |  38.31 |   3.34 | SW     |
|  3 |    3 |  77.65 |   5.7  |   65.76 |    4.77 |  40.04 |   3.52 |  69.17 |   3.4  |  35.78 |   3.41 | SW     |
|  4 |    4 |  62.8  |   4.84 |   52.09 |    3.73 |  33.95 |   2.72 |  56.27 |   2.96 |  31.88 |   3.13 | SW     |

The shape of the df is (420, 12). After na values are dropped this further reduces to (413,12).


The data is then split into two pieces, the labels (Type) in one df.


|    | type   |     
|---:|:-------|
|  0 | SW     |
|  1 | SW     |
|  2 | SW     |
|  3 | SW     |
|  4 | SW     |







and all of the other pieces of relevant data into another. We would not want the model to know the answers.


|    |   huml |   humw |   ulnal |   ulnaw |   feml |   femw |   tibl |   tibw |   tarl |   tarw |
|---:|-------:|-------:|--------:|--------:|-------:|-------:|-------:|-------:|-------:|-------:|
|  0 |  80.78 |   6.68 |   72.01 |    4.88 |  41.81 |   3.7  |   5.5  |   4.03 |  38.7  |   3.84 |
|  1 |  88.91 |   6.63 |   80.53 |    5.59 |  47.04 |   4.3  |  80.22 |   4.51 |  41.5  |   4.01 |



Then a pair plot was loaded. 

![alt text](Pair_plot_all.png "Pair plot of everything")

The above pair plot shows a lot of covariance and maybe coliniearity. Which makes some sense, if a bird has big hip bones it probably has big leg bones too. But this might not be so great for training a model, and we might want to exclude some of those features.  So far the variables 'huml','humw', 'ulnal','ulnaw', 'tarl' look the most promising being more randomly scattered than say 'femw'. But we should look at a few more pieces of evidence before we fully decide to drop or keep certain variables.

## Box plots to check for outliers

Box plots were plotted against certain features for simple and straightforward outlier detection. And it is very obvious that there are outliers. And they will be removed so they do not skew the data too much.

![alt text](Box_plot_To_check_for_outliers.png "Box plot to check for outliers")

Now that the data is a little bit more cleaned up. I will plot a correlation matrix, which pairs well with the pair plot above. It helps show what variables are highly correlated and how strong that relationship is. Correlation is how closely related certain variables are. There are only a few variables that do not have very strong relationships. Humerus length, Humerus Width, Ulnal length and width and tarsal length all have low correlation values. The Covariance matrix also has values that are very high and shows a definite need for some feature selection. 


![alt text](Correlation_Matrix_before_choosing.png "Pair plot of everything")




![alt text](Covariance_Matrix_before_choosing_(2).png)

To help illustrate the point further, at least a little bit. I ran an analysis on the full data set using the clustering algorithm and have the generated results. 
There are two variables in table below. One is silhouette score or the silhouette coefficient. Which ranges from -1 to 1, 1 being the best value and -1 being the worst.
It is the mean distance between clusters, and the mean distance to the nearest cluster that is not part of that samples cluster. Given by the formula:


\begin{pmatrix}
   b - a\\ \hline
   max(a,b)
\end{pmatrix}  


<h5 align="center">a = intra cluster distance </h5> 
<h5 align="center">b = mean nearest intracluster distance</h5>


|    |    silhouette_score |   rand_score |
|---:|---------:|-------------:|
|  0 | 0.374319 |     0.239816 |



And since I have the ground truth, which are my labels (the bird types) I can run a metric called the adjusted Rand Index. Which uses the actual cluster assignment to provide the score. It is a pairwise measure which computes the number of same or different clusters in the predicted and true clusterings. 
A value of 0.0 means the labels are random independent of the cluster amounts and 1.0 when the clusters are completely identical. So a 1 is good and 0 is not as good.

Rand_score_Calculation : <b>ARI = (RI - Expected_RI) / (max(RI) - Expected_RI)</b> ** thanks scikit learn 

And the obtained result is not that great.Removing some features might help.

Only a few features were selected.And they had a pair plot generated. 

It is very clearly visible that the data looks much more scattered, which might be a good sign. Time to move onto the next part.



![alt text](Pair_plot_reduced.png "pair plot of just a few features")

### Pre Processor Selection

To scale or not to scale? One of life's big questions if you need to proccess data. 
A great way to tell is by looking at the covariance matrix.

 Which measures the extent to which two variables er vary together.
 And the values in this matrix are very high.


![alt text](Covariance_Matrix_to_show_need_scaling.png "pair plot of just a few features")
![alt text](Correlation_Matrix_before_scaling_but_after_feature_selection.png)


I had several preprocessors to choose from. 

![alt text](For_final_project_unsupervised.jpg "selection of features")

- Standard Scaler - Sets the mean to 0 and variance to 1.

- MinMiaxScaler - Transforms features by scaling to a given range.

- MaxAbsScaler - Scales Each feature by maximum abosulte value.

- RobustScaler- Removes means and scales each to a Quantile range.

- Normalizer - Scales inputs individually to length of data.


So I ran all of these against my data set and their silhouette scores are below. 


![alt text](Selection_preprocessor_results.png)

The silhouette score improved a fair amount from the base model. The adjusted rand score improved only slightly.
And the thing about Kmeans is the results will vary slightly every time. 
When I plotted the results for each pre processing step:

<b> Standard Scaler </b>

![alt text](Standard_Scaler_pre_processing.png)

<b> MinMax Scaler </b>

![alt text](MinMax_Scaler_pre_processing.png)

<b> MaxABS Scaler </b>


![alt text](MaxABS_Scaler_pre_processing.png)

<b> Robust Scaler </b>


![alt text](Robust_Scaler_pre_processing.png)


The below covariance matrix helps highlight this.

![alt text](Covariance_matrix_after_scaling.png)




Each type of pre processor led to a similiar improvement no matter the choices. I wanted to see if another second preprocessor would help improve them further.

I chose Kernel PCA and PCA.
- Kernel PCA uses kernels for dimensionality reduction.

- PCA projects data onto a lower dimensional space.


<b> PCA Results </b>


![alt text](Regular_PCA.png)


<b> KernelPCA </b>

![alt text](KernelPCA.png)

I also ran different kernel types, rbf, linear and cosine. And the results were fairly similiar accross the board.

In [2]:
data50=pd.read_csv('Data_frame_for_plot_of_kernels')


In [3]:
data50.hvplot(x='Run', y=["rbf","linear","cosine"], kind='scatter')

:NdOverlay   [Variable]
   :Scatter   [Run]   (value)

The cosine Kernel performed the best overall. So I chose that to be my kernel. 

I also tried different values for alpha and gamma for my kernels and all of them performed similiarly.


![alt text](Kernel_PCA_gamma001.png)

In [5]:
data70=pd.read_csv('Data_frame_for_plot_of_alpha_gamma')
data70.hvplot(x='Run', y=["Gamma0.001","Gamma1","Gamma10","Gamma100","Alpha0.01","Alpha5"], kind='scatter')

:NdOverlay   [Variable]
   :Scatter   [Run]   (value)

After selecting parameters that seemed to work well I chose best parameters for the clusterer.
I chose kmeans++, and the elkan algorithm. 
It was then time to fit my model and see how well it worked.

## Results and conclusion

No matter what parameters I changed, or how I split my data. My model performance was average at best.

![alt text](Final_clustering.png)

The first prediction or so was usually pretty high for accuracy, but then it quickly dipped and stayed low. 
This leads me to believe that Kmeans was not an appropriate choice for my data set. But I have come away from this project with stronger knowledge of Kmeans and how to apply it. 
I did notice with repetition that the model also continually misclassified category 2. Which means the features are not different enough even when scaled to be able to be distinguished from each other. 
I plotted some results obtained after that feature was dropped. But they are not too different from my original results.
This problem was probably better suited for a supervised learning algorithm. 

In [6]:
data70=pd.read_csv('result_values_final')
data70.hvplot(x='Run', y=["elkan"], kind='scatter')

:Scatter   [Run]   (elkan)

Origial Results above, after removal results below. 

In [7]:
data95=pd.read_csv("full_items_result_values_final")
data95.hvplot(x='Run', y="elkan", kind='scatter')

:Scatter   [Run]   (elkan)

While the clustering here looks a little  better. The actual accuracy of the model ended up being  poor. 

![alt text](Final_clustering_after_SO_removed.png)